In [ ]:
# hide
# skip
!git clone https://github.com/benihime91/gale # install gale on colab
!pip install -e "gale[dev]"

In [ ]:
# default_exp classification.modelling.meta_arch.common

In [ ]:
# hide
%load_ext nb_black
%load_ext autoreload
%autoreload 2
%matplotlib inline

<IPython.core.display.Javascript object>

In [ ]:
# hide
import warnings

from nbdev.export import *
from nbdev.showdoc import *

warnings.filterwarnings("ignore")

<IPython.core.display.Javascript object>

# Meta Architectures : Generalized Image Classifier 
> Default Model Architecture for Image Classification

In [ ]:
# export
import logging
from typing import *

import torch
from omegaconf import DictConfig, OmegaConf
from pytorch_lightning.core.memory import get_human_readable_count
from torch.nn import Module

from gale.classification.modelling.backbones import ImageClassificationBackbone
from gale.classification.modelling.build import build_backbone, build_head
from gale.classification.modelling.heads import ImageClassificationHead
from gale.core.classes import GaleModule
from gale.core.nn.shape_spec import ShapeSpec

_logger = logging.getLogger(__name__)

<IPython.core.display.Javascript object>

In [ ]:
# export
class GeneralizedImageClassifier(GaleModule):
    """
    A General Image Classifier. Any models that contains the following 2 components:
    1. Feature extractor (aka backbone)
    2. Image Classification head (Pooling + Classifier)
    """

    def __init__(
        self,
        backbone: ImageClassificationBackbone,
        head: ImageClassificationHead,
    ):
        """
        Arguments:
        1. `backbone`: a `ImageClassificationBackbone` module, must follow gale's backbone interface
        2. `head`: a head containg the classifier. and the pooling layer, must be an instance of
        `ImageClassificationHead`.
        """
        super(GeneralizedImageClassifier, self).__init__()
        self.backbone = backbone
        assert isinstance(backbone, ImageClassificationBackbone)
        self.head = head
        assert isinstance(head, ImageClassificationHead)

    def forward(self, batched_inputs: torch.Tensor) -> torch.Tensor:
        """
        Runs the batched_inputs through `backbone` followed by the `head`.
        Returns a Tensor which contains the logits for the batched_inputs.
        """
        out = self.backbone(batched_inputs)
        out = self.head(out)
        return out

    @classmethod
    def from_config_dict(cls, cfg: DictConfig):
        """
        Instantiate the Meta Architecture from gale config
        """
        # fmt: off
        
        if not hasattr(cfg.model, "backbone"):
            _logger.error("Configuration for model backbone not found")
            raise ValueError
            
        if not hasattr(cfg.model, "head"):
            _logger.error("Configuration for model head not found")
            raise ValueError
            
        input_shape = ShapeSpec(cfg.input.channels, cfg.input.height, cfg.input.width)
        _logger.debug(f"Inputs: {input_shape}")
        
        backbone = build_backbone(cfg, input_shape=input_shape)
        param_count = get_human_readable_count(sum([m.numel() for m in backbone.parameters()]))
        _logger.debug('Backbone {} created, param count: {}.'.format(cfg.model.backbone.name, param_count))
        
        head = build_head(cfg, backbone.output_shape())
        param_count = get_human_readable_count(sum([m.numel() for m in head.parameters()]))
        _logger.debug('Head {} created, param count: {}.'.format(cfg.model.head.name, param_count))
        
        kwds = {"backbone": backbone, "head": head}
        
        instance = cls(**kwds)
        instance._cfg = OmegaConf.to_container(cfg.model, resolve=True)
        instance.input_shape = input_shape
        param_count = get_human_readable_count(sum([m.numel() for m in instance.parameters()]))
        _logger.info("Model created, param count: {}.".format(param_count))
        # fmt: on
        return instance

    def build_param_dicts(self):
        """
        Builds up the Paramters dicts for optimization.
        """
        backbone_params = self.backbone.build_param_dicts()
        head_params = self.head.build_param_dicts()
        return backbone_params + head_params

    def get_lrs(self) -> List:
        """
        Returns a List containining the Lrs' for
        each parameter group. This is required to build schedulers
        like `torch.optim.lr_scheduler.OneCycleScheduler` which needs
        the max lrs' for all the Param Groups.
        """
        lrs = []

        for p in self.build_param_dicts():
            lrs.append(p["lr"])
        return lrs

<IPython.core.display.Javascript object>

This model architecture will work for most common computer vision fietuning use case. We take a `backbone` and `classifier`. We run the input through the backbone to extract the feature_maps which are then used by the classifier to given predictions on the Input. The paramters dicts for optimization are generated by the `backbone` and the `head` itself.

> Note: For advanced use cases you might want to create a model. A model muse inherit from `GaleModule` and be registered in `META_ARCH_REGISTRY`. Your model should also have the following methods to work in the Gale ecosystem.

In [ ]:
show_doc(GeneralizedImageClassifier.from_config_dict)

<h4 id="GeneralizedImageClassifier.from_config_dict" class="doc_header"><code>GeneralizedImageClassifier.from_config_dict</code><a href="__main__.py#L35" class="source_link" style="float:right">[source]</a></h4>

> <code>GeneralizedImageClassifier.from_config_dict</code>(**`cfg`**:`DictConfig`)

Instantiate the Meta Architecture from gale config

<IPython.core.display.Javascript object>

In [ ]:
show_doc(GeneralizedImageClassifier.forward)

<h4 id="GeneralizedImageClassifier.forward" class="doc_header"><code>GeneralizedImageClassifier.forward</code><a href="__main__.py#L26" class="source_link" style="float:right">[source]</a></h4>

> <code>GeneralizedImageClassifier.forward</code>(**`batched_inputs`**:`Tensor`)

Runs the batched_inputs through `backbone` followed by the `head`.
Returns a Tensor which contains the logits for the batched_inputs.

<IPython.core.display.Javascript object>

In [ ]:
show_doc(GeneralizedImageClassifier.build_param_dicts)

<h4 id="GeneralizedImageClassifier.build_param_dicts" class="doc_header"><code>GeneralizedImageClassifier.build_param_dicts</code><a href="__main__.py#L71" class="source_link" style="float:right">[source]</a></h4>

> <code>GeneralizedImageClassifier.build_param_dicts</code>()

Builds up the Paramters dicts for optimization.

<IPython.core.display.Javascript object>

`Meta_Arch`'s can also be instatiated via a appropriate config file. Let's see how ..

In [ ]:
from dataclasses import dataclass, field
from omegaconf import MISSING, OmegaConf
from gale.classification.modelling.backbones import ResNetBackbone
from gale.classification.modelling.heads import FastaiHead

<IPython.core.display.Javascript object>

For a meta_arch we first need to create the configurations for the `Backbone` and the `Head` of the model. These 
must be registerd in `IMAGE_CLASSIFICATION_BACKBONES` and `IMAGE_CLASSIFICATION_HEADS` Registy respectively. The instances are automatically instiated by the `GeneralizedImageClassifier` meta_arch.

In [ ]:
@dataclass
class BackboneConf:
    model_name: str = "resnetblur18"
    act: Any = None
    lr: Any = 2e-04
    lr_div: Any = 10
    wd: Any = 1e-02
    freeze_at: int = 0
    pretrained: bool = False
    drop_block_rate: Any = None
    drop_path_rate: Any = None
    bn_tf: bool = False


@dataclass
class HeadConf:
    num_classes: int = MISSING
    act: str = "ReLU"
    lin_ftrs: Any = None
    ps: Any = 0.5
    concat_pool: bool = True
    first_bn: bool = True
    bn_final: bool = False
    lr: float = 2e-03
    wd: float = 1e-02
    filter_wd: bool = False

<IPython.core.display.Javascript object>

We also need a few more things and the config must be composed in a gale config style. We need the definitions of the input like channels, height and weight. So let's compose these -

In [ ]:
b_args = OmegaConf.structured(BackboneConf())
h_args = OmegaConf.structured(HeadConf(num_classes=10))

# Backbone config
b = OmegaConf.create()
b.name = "ResNetBackbone"
b.init_args = b_args

# Head config
h = OmegaConf.create()
h.name = "FastaiHead"
h.init_args = h_args

i = OmegaConf.create()
i.channels = 3
i.height = 224
i.width = 224

m = OmegaConf.create()
m.backbone = b
m.head = h

<IPython.core.display.Javascript object>

In [ ]:
conf = OmegaConf.create(dict(input=i, model=m))

<IPython.core.display.Javascript object>

In [ ]:
m = GeneralizedImageClassifier.from_config_dict(conf)

assert isinstance(m, GeneralizedImageClassifier)
assert isinstance(m.backbone, ResNetBackbone)
assert isinstance(m.head, FastaiHead)

<IPython.core.display.Javascript object>

In [ ]:
# collapse-output
print(m)

GeneralizedImageClassifier(
  (backbone): ResNetBackbone(
    (stem): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Sequential(
        (0): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
        (1): BlurPool2d(
          (padding): ReflectionPad2d([1, 1, 1, 1])
        )
      )
    )
    (stages): Sequential(
      (0): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act1): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

<IPython.core.display.Javascript object>

In [ ]:
shape = (m.input_shape.channels, m.input_shape.height, m.input_shape.width)
inp = torch.randn(2, *shape)
o = m(inp)

<IPython.core.display.Javascript object>

## Export-

In [ ]:
# hide
notebook2script()

Converted 00_core.utils.logger.ipynb.
Converted 00a_core.utils.visualize.ipynb.
Converted 00b_core.utils.structures.ipynb.
Converted 01_core.nn.utils.ipynb.
Converted 01a_core.nn.losses.ipynb.
Converted 02_core.nn.optim.optimizers.ipynb.
Converted 02a_core.nn.optim.lr_schedulers.ipynb.
Converted 03_core.classes.ipynb.
Converted 04_classification.modelling.backbones.ipynb.
Converted 04a_classification.modelling.heads.ipynb.
Converted 04b_classification.modelling.meta_arch.common.ipynb.
Converted 04b_classification.modelling.meta_arch.vit.ipynb.
Converted 05_classification.data.common.ipynb.
Converted 05a_classification.data.transforms.ipynb.
Converted 05b_classification.data.build.ipynb.
Converted 06_classification.task.ipynb.
Converted 07_collections.pandas.ipynb.
Converted 07a_collections.callbacks.notebook.ipynb.
Converted 07b_collections.callbacks.ema.ipynb.
Converted index.ipynb.


<IPython.core.display.Javascript object>